In [1]:
import chardet
 
with open('msr_training.txt', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']
    print("Detected encoding:", encoding)


Detected encoding: GB2312


In [2]:
# coding:utf-8
import sys
import gensim
import sklearn
import numpy as np
import codecs
import jieba

In [3]:
#读取语料库
with open("msr_training.txt", 'r',encoding='gb2312', errors='ignore') as cf:
    docs = cf.readlines()
    print(len(docs))
print(docs[10])

86924
“  那阵子  ，  条件  虽  艰苦  ，  可  大家  热情  高着  呢  ，  什么  活  都  抢  着  干  ，  谁  都  争  着  多  作  贡献  。



In [4]:
#为句子做标记
train_data = []       #建一个空列表存储数据
for i, text in enumerate(docs):    #遍历docs的内容     enumerate函数用于返回索引i和text的值
    word_list = text.split(' ')    #将text用空格进行分词会赋予word_list
    l = len(word_list)      #将计算word_list的长度赋予变量l
    word_list[l-1] = word_list[l-1].strip()    #将word_list列表的最后一个元素进行strip操作，去除两边的空白符合
    document = gensim.models.doc2vec.TaggedDocument(word_list, tags=[i]) #创建TaggedDocument对象，其中word_list是单词列表，tags是一个包含当前文档索引的列表
    train_data.append(document)  #将document对象添加到训练数据列表中

In [5]:
#定义模型
model = gensim.models.doc2vec.Doc2Vec(train_data,
                                     min_count=1, #忽略词频小于此值的所有单词，为1就说明不忽略任何单词
                                     window = 3,  #训练单词上下文时考虑的窗口大小，这里设置上下3个单词
                                     vector_size = 256, #文档向量的维度大小256
                                     negative=10, #如果大于0，则使用负采样，采取10个负样本
                                     workers=4,  #训练模型时使用4个线程数
                                     alpha = 0.001, #设置学习率为0.001
                                     min_alpha=0.001) #最小学习率为0.001

In [6]:
#训练模型
model.train(train_data, 
             total_examples=model.corpus_count, 
             epochs=10)
model.save('model_msr')

In [7]:
#定义计算余弦相似度函数
def sim_cal(vector_1, vector_2):
    vector1_mod = np.sqrt(vector_1.dot(vector_1))
    vector2_mod = np.sqrt(vector_2.dot(vector_2))
    if vector2_mod != 0 and vector1_mod != 0:
        similarity = (vector_1.dot(vector_2)) / (vector1_mod * vector2_mod)
    else:
        similarity = 0
    return similarity

In [8]:
def doc2vec(file_name, model):
    doc = [w for x in codecs.open(file_name, 'r', 'utf-8').readlines() for w in jieba.cut(x.strip())]
    doc_vec_all = model.infer_vector(doc)
    return doc_vec_all

In [12]:
#查看文档1
f = open("fewf.txt", "r", encoding="utf-8")
web_text = f.read()
print("网页文本1：")
print(web_text)
print()
f.close()
#查看文档2
f = open("ewfe.txt", "r", encoding="utf-8")
web_text = f.read()
print("网页文本2：")
print(web_text)
f.close()


网页文本1：
词向量（Word embedding），又叫Word嵌入式自然语言处理（NLP）中的一组语言建模和特征学习技术的统称，其中来自词汇表的单词或短语被映射到实数的向量。 从概念上讲，它涉及从每个单词一维的空间到具有更低维度的连续向量空间的数学嵌入。


网页文本2：
词向量（Word Vector）是将自然语言中的词语转换成计算机能够理解的数学向量的过程。在自然语言处理（NLP）和深度学习领域，词向量是表示词语特征的一种方式，它使得计算机能够处理和理解文本数据。

传统的词向量表示方法是独热编码（One-Hot Encoding），即每个词在向量中对应一个位置，该位置上的值为1，其余位置上的值为0。然而，这种方法存在一些问题，如维度灾难（随着词汇量的增加，向量维度急剧增大）和语义鸿沟（无法体现词语之间的语义关系）。

为了克服这些问题，人们提出了分布式表示（Distributed Representation）的方法，其中最具代表性的是Word2Vec。Word2Vec通过训练神经网络模型来学习词语的向量表示，使得具有相似上下文的词语在向量空间中接近。这种方法不仅降低了向量的维度，还能捕捉到词语之间的语义和语法关系。

词向量的应用非常广泛，包括文本分类、情感分析、机器翻译、问答系统等。通过词向量，计算机可以更好地理解和处理文本数据，从而实现更高级的自然语言处理任务。

总之，词向量是将自然语言中的词语转换成数学向量的过程，它使得计算机能够处理和理解文本数据，并广泛应用于各种自然语言处理任务中。


In [13]:
#计算文档相似度
model = gensim.models.doc2vec.Doc2Vec.load('model_msr')
p1 = 'fewf.txt'
p2 = 'ewfe.txt'
p1_doc2vec = doc2vec(p1, model)
p2_doc2vec = doc2vec(p2, model)
print(sim_cal(p1_doc2vec, p2_doc2vec))


0.84737736
